In [1]:
from pydantic_ai import Agent, RunContext
from dotenv import load_dotenv
import nest_asyncio

nest_asyncio.apply()

load_dotenv()

True

In [37]:
from pydantic import BaseModel
from typing import Literal

def create_agent_output_class(list_of_next_agents: list[str]):
    class MASAgentResponse(BaseModel):
        response: str
        goto: Literal[*list_of_next_agents]  # Unpacks list into Literal values

        def __repr__(self):
            return (
                f"MASAgentResponse(response={self.response}, next_agent={self.goto})\n"
                f"  [next_agent must be one of: {list_of_next_agents}]"
            )

    return MASAgentResponse

In [38]:
so_next_agents = ['user', 'drill_creation_agent', 'curriculum_tracking_agent']

supervisor_agent_system_prompt = """
You are the supervisor agent of a multi-agent system assisting with Spanish language learning.

As the supervisor agent, you are coordinating the actions of the other agents to perform tasks, and communicate the results to the user.
You are always delegating or aggregating tasks, never doing the work yourself.

If you are given a task, you will delegate it to the appropriate agent, by instructing said agent to perform the task.
If you are given the results of a task, you will aggregate the results, and communicate the results to the user.

Here is a list of the agents you can use:
drill_creation_agent: creates drills based on a sprecific requested drill type.
curriculum_tracking_agent: tracks the user's language learning progress, and can suggest next steps in the curriculum.
"""

supervisor_agent = Agent(  
    'gpt-4o-mini',
    system_prompt=supervisor_agent_system_prompt,
    output_type=create_agent_output_class(so_next_agents)
)



In [42]:
drill_creation_next_agents = ['supervisor_agent', 'curriculum_tracking_agent']

drill_creation_agent_system_prompt = """
You are a drill creation agent. You are responsible for creating drills based on the requested drill type.

Here is a list of the drill types you can create:  
Grammar drills: drills that focus on a specific grammar concept.
Translation drills: drills that focus on translating a specific text from English to Spanish or from Spanish to English.
Vocabulary drills: drills that focus on a specific vocabulary word.

To create the drill, you will need to have the following information:
- The drill type
- The current user's level of Spanish proficiency

If you get a specific drill type and user level proficiency, create a suitable drill and present it to the supervisor agent.
"""

drill_creation_agent = Agent(
    'gpt-4o-mini',
    system_prompt=drill_creation_agent_system_prompt,
    output_type=create_agent_output_class(drill_creation_next_agents)
)

In [43]:
curriculum_tracking_next_agents = ['supervisor_agent', 'drill_creation_agent']

curriculum_tracking_agent_system_prompt = """
You are the curriculum tracking agent. You are responsible for tracking the user's language learning progress, and suggesting next steps in the curriculum.
"""

curriculum_tracking_agent = Agent(
    'gpt-4o-mini',
    system_prompt=curriculum_tracking_agent_system_prompt,
    output_type=create_agent_output_class(curriculum_tracking_next_agents)
)

@curriculum_tracking_agent.tool_plain
def get_grammar_concept_proficiency() -> str:
    return """
    Current user's level of Spanish proficiency in grammar:
    - Verb conjugation: not yet started
    - Past tense: not yet started
    - Present perfect: not yet started
    - Present participle: not yet started
    - Preterite: not yet started
    - Future: not yet started
    - Conditional: not yet started
    - Imperative: not yet started
    """

In [44]:
input = 'I would like to practice my Spanish. I would like to drill my next grammar concept.'
agent_mapping = {
    'supervisor_agent': supervisor_agent,
    'drill_creation_agent': drill_creation_agent,
    'curriculum_tracking_agent': curriculum_tracking_agent
}
current_agent = 'supervisor_agent'

print(f"Starting {current_agent} with input: {input}")

history = []

def get_prompt(history, initial_input=input) -> str:
    if len(history) == 0:
        return f"The user has given the following request: {initial_input}."
    
    # Else
    new_prompt = f"The user has given the following initial request: {initial_input}.\n The following is the history of responses of the various agents that have acted so far on this request:\n"
    for agent, response, next_agent in history:
        new_prompt += f"Agent {agent} instructed agent {next_agent}: {response}\n"

    return new_prompt
        
for i in range(10):
    prompt = get_prompt(history)
    print(f"Agent {current_agent} is instructed: {prompt}")
    
    result = agent_mapping[current_agent].run_sync(prompt)

    print(f"{current_agent} returned:")
    print(f"Response: {result.output.response}")
    print(f"Next agent: {result.output.goto}")
    print("--------------------------------")

    if result.output.goto == 'user':
        break

    history.append((current_agent, result.output.response, result.output.goto))
    current_agent = result.output.goto

Starting supervisor_agent with input: I would like to practice my Spanish. I would like to drill my next grammar concept.
Agent supervisor_agent is instructed: The user has given the following request: I would like to practice my Spanish. I would like to drill my next grammar concept..
supervisor_agent returned:
Response: I will delegate the task of creating a drill for your next grammar concept to the drill_creation_agent. Please wait a moment while I do that.
Next agent: drill_creation_agent
--------------------------------
Agent drill_creation_agent is instructed: The user has given the following initial request: I would like to practice my Spanish. I would like to drill my next grammar concept..
 The following is the history of responses of the various agents that have acted so far on this request:
Agent supervisor_agent instructed agent drill_creation_agent: I will delegate the task of creating a drill for your next grammar concept to the drill_creation_agent. Please wait a moment

Agent(model=OpenAIModel(), name='drill_creation_agent', end_strategy='early', model_settings=None, output_type=<class '__main__.create_agent_output_class.<locals>.MASAgentResponse'>, instrument=None)